In [1]:
!pip install qiskit qiskit-finance qiskit-optimization numpy pandas matplotlib yfinance scipy

  Using cached qiskit-2.0.0-cp39-abi3-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached qiskit_optimization-0.6.1-py3-none-any.whl.metadata (8.6 kB)
  Using cached qiskit_algorithms-0.3.1-py3-none-any.whl.metadata (4.2 kB)
  Preparing metadata (setup.py) ... done
  Using cached docplex-2.29.241.tar.gz (645 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
Using cached qiskit-2.0.0-cp39-abi3-macosx_11_0_arm64.whl (6.0 MB)
Using cached qiskit_optimization-0.6.1-py3-none-any.whl (167 kB)
Using cached networkx-3.4.2-py3-none-any.whl (1.7 MB)
Using cached qiskit_algorithms-0.3.1-py3-none-any.whl (310 kB)
  Created wheel for docplex: filename=docplex-2.29.241-py3-none-any.whl size=685423 sha256=b67a65ea60a6dc93c3eb703056ea946f99c135c32a8a0a1b1ab71a62564d0f04
  Stored in directory: /Users/dhairy

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from qiskit import Aer
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo

ImportError: cannot import name 'Aer' from 'qiskit' (/opt/homebrew/lib/python3.11/site-packages/qiskit/__init__.py)